# Test for WAV conversion

Everything is mostly preset. You only have to adjust the output path. 

This notebook just tests that everything is fine with the WAV conversion after a new implementation. It is very hard coded, if you are making changes make sure you are doing them throughout this short notebook.

In [1]:
# solve issue wi\th autocomplete
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import os
import sys
import time
import glob
from IPython import display

from soundfile import SoundFile
from natsort import natsorted

import matplotlib.pyplot as plt
from matplotlib import cm
import cartopy.crs as ccrs

import numpy as np
import pandas as pd
from obspy import read, Trace
from obspy.signal.tf_misfit import plot_tfr

# the file src_sac2wav contains written functions for exporting sac to wav files
from src_sac2wav import *

## Download the data

This function downlaods 2 sets of data:
    
       - continuous and day based
       - event based

After this the code automatically converts to WAV files. There will be three outputs:

       - one in the events folder
       - two in the continuous folder (for day and continuous conversion)

### Downloads data for testing the codes

In [3]:
test_data_path = '/mnt/seismodata2/MT/_OUTPUT_/TEST_DATA_SAC2WAV'

If you downloaded this data before and you just want to test the conversion you do not need to download the next cell...

In [4]:
# %% capture
command_event = f'obspyDMT --datapath {test_data_path}/test_event --min_date 2021-02-01 --max_date 2021-05-03 --waveform_format sac --data_source all --pre_process process_unit_wav --min_mag 8.0 --max_mag 10 --event_catalog NEIC_USGS --sta DSB --cha B* --net EI --preset 300 --offset 7200 --instrument_correction --req_parallel --req_np 4 --parallel_process --process_np 4' 
wav_save, save_fig_path = generate_output_folders('event', f'{test_data_path}/test_event')
!{command_event}

command_continuous = f'obspyDMT --datapath {test_data_path}/test_continuous --min_date 2021-03-03 --max_date 2021-03-06 --waveform_format sac --data_source IRIS --pre_process process_unit_wav --continuous --sta RAO --cha B* --net IU --instrument_correction --req_parallel --req_np 4 --parallel_process --process_np 4'
wav_save, save_fig_path = generate_output_folders('continuous', f'{test_data_path}/test_continuous')
!{command_continuous}


/mnt/store/mariat/anaconda3/envs/py37/lib/python3.7/site-packages/obspy/clients/arclink/__init__.py:178: ObsPyDeprecationWarning: The ArcLink protocol will be deprecated in the near future. Please, use the client contacting the routing service provided by EIDA: https://docs.obspy.org/packages/obspy.clients.fdsn.html#basic-routing-clients-usage
  warnings.warn(msg, category=ObsPyDeprecationWarning)
--------------------------------------------------------------------------------
		   obspyDMT (obspy Data Management Tool)

	Python Toolbox for Retrieving, Processing and Management of
			Large Seismological Datasets

:copyright:
The ObsPy Development Team (devs@obspy.org)

Developed by Kasra Hosseini
email: kasra.hosseinizad@earth.ox.ac.uk

:license:
GNU General Public License, Version 3
(http://www.gnu.org/licenses/gpl-3.0-standalone.html)
--------------------------------------------------------------------------------


obspyDMT primary mode: event_based


Waveforms will be retrieved from


DONE with Event: continuous2
Time: 0:00:00.069120

metadata mode
initializing files and directories...DONE
check the availability: IRIS

availability for event: 3/3 ---> DONE
Time for checking the availability: 0:00:00.416567
[INFO] requested data sources:
IRIS 

IRIS -- event: /mnt/seismodata2/MT/_OUTPUT_/TEST_DATA_SAC2WAV/test_continuous/continuous3

DONE with Event: continuous3
Time: 0:00:00.095623
[LOCAL] Path:
/mnt/seismodata2/MT/_OUTPUT_/TEST_DATA_SAC2WAV/test_continuous/continuous1
[INFO] update station_event file...
[1/6] start processing: IU.RAO.00.BH1
[5/6] start processing: IU.RAO.10.BH2
[3/6] start processing: IU.RAO.00.BHZ
[4/6] start processing: IU.RAO.10.BH1
[2/6] start processing: IU.RAO.00.BH2
[6/6] start processing: IU.RAO.10.BHZ


All 3 processes are finished...
[LOCAL] Path:
/mnt/seismodata2/MT/_OUTPUT_/TEST_DATA_SAC2WAV/test_continuous/continuous2
[INFO] update station_event file...
[1/6] start processing: IU.RAO.00.BH1
[5/6] start processing: IU.RAO.10.BH2
[3/6] 

## Convert to WAV

In [5]:
df_continous = read_station_information(f'{test_data_path}/test_continuous', '*')
df_event = read_station_information(f'{test_data_path}/test_event', '*')

# Continuous data test
wav_save, save_fig_path = generate_output_folders('continuous', f'{test_data_path}/test_continuous')
export_continuous(df_continous, True, f'{test_data_path}/test_continuous', 'instr', 
                  ['2021-03-03', 3], '*', '*', 48000, 'PCM_24', wav_save, True)
print('-------------------------------------------------------------------------------------------')
# Day data test
wav_save, save_fig_path = generate_output_folders('day', f'{test_data_path}/test_continuous')
export_day(df_continous, True, f'{test_data_path}/test_continuous', 'instr', 
                  ['2021-03-03', 3], '*', '*', 48000, 'PCM_24', wav_save, True)
print('-------------------------------------------------------------------------------------------')
# Event data test
wav_save, save_fig_path = generate_output_folders('event', f'{test_data_path}/test_event')
export_event(df_event, True, f'{test_data_path}/test_event', 'instr', 
                  ['20210304_192833.a'], '*', '*', 48000, 'PCM_24', wav_save, True)


Days to process:
	['continuous1', 'continuous2', 'continuous3']
Working on station: RAO
Location: 00
Channel Group: ['BH1', 'BH2', 'BHZ', 'BDH']

	Channel BH1.
		Folder continuous1
			Adding IU.RAO.00.BH1.
		Folder continuous2
			Adding IU.RAO.00.BH1.
		Folder continuous3
			Adding IU.RAO.00.BH1.
	Length of trace: 10368000

	Channel BH2.
		Folder continuous1
			Adding IU.RAO.00.BH2.
		Folder continuous2
			Adding IU.RAO.00.BH2.
		Folder continuous3
			Adding IU.RAO.00.BH2.
	Length of trace: 10368000

	Channel BHZ.
		Folder continuous1
			Adding IU.RAO.00.BHZ.
		Folder continuous2
			Adding IU.RAO.00.BHZ.
		Folder continuous3
			Adding IU.RAO.00.BHZ.
	Length of trace: 10368000

	Channel BDH.
		Folder continuous1
			--Missing: continuous1/proc_instr/*RAO*00*BDH
		Folder continuous2
			--Missing: continuous2/proc_instr/*RAO*00*BDH
		Folder continuous3
			--Missing: continuous3/proc_instr/*RAO*00*BDH
			Adding a zero trace for the hydrophone channel...
	Normalizing channelwise after mergin